In [6]:
# Загружаем библиотеки  
from fastai.vision.all import *
import os
from imageio.v2 import imread

from fastcore.all import *


In [5]:

# Функция проверяющая файл изображения. В случае если изображение не распознано, как птица - переносит этот файл в папку dir_to_dump и переименовывает его.
def not_a_bird_check(file_path,dir_to_dump,ind):
    """
        Проверяет, является ли изображение из файла птицей. В случае если изображение не распознано, как птица - переносит этот файл в папку dir_to_dump и переименовывает его
    Args:
        file_path (str): Путь к файлу, который нужно проверить
        dir_to_dump (str): Путь к папке в которую перемещаются не птицы,
        ind (int): Индекс
    Return:    
        Возвращяет 0 если изображение не является птицей и 1 если нет
    """

    
    
    img = imread(file_path)
    try:
        ai_class,__,__ = bird_not_bird_learner.predict(img)                # bird_not_bird_learner загркжается в основной части кода
        if ai_class!='0':                                                  # если не птица
            ext=str(file_path).split('.')[-1]                              # вычленяем из пути файла расширение файла
            name=os.path.split(file)[-1]                                   # вычленяем из пути файла имя файла
            newname='from__'+os.path.split(file)[0].replace('\\','__')+'__'+сlass_dict[ai_class]+'__'+str(ind)+'.'+ext # создаем новое имя файла в котором будет указание на папку из которой он взят, класс который ему присвоен, индекс, 
            shutil.move(file_path, f'{dir_to_dump}/{name}')                # переносим файл в папку dir_to_dump 
            os.rename(f'{dir_to_dump}/{name}',f'{dir_to_dump}/{newname}')  # переименовываем перенесенный файл
            #print('не птица')
            return(1)                                                      # если файл отбракован - возвращаем 1, если нет -0
    except:
        ext=str(file_path).split('.')[-1]                                  # вычленяем из пути файла расширение файла
        name=os.path.split(file)[-1]                                       # вычленяем из пути файла имя файла
        shutil.move(file_path, f'{dir_to_dump}/{name}')                    # переносим файл в папку dir_to_dump 
        newname='trubl_'+'from_'+os.path.split(file)[0].replace('\\','__')+'__'+str(ind)+'.'+ext  # создаем новое имя файла в котором будет указание на папку из которой он взят, проблемность файла, и индекс
        os.rename(f'{dir_to_dump}/{name}',f'{dir_to_dump}/{newname}')      # переименовываем перенесенный файл
        return(1)                                                          # если файл отбракован - возвращаем 1, если нет -0
    return(0)                                                              # если файл отбракован - возвращаем 1, если нет -0
   

In [ ]:
def birds_check(dir_to_control):
    """
        Пробегает по всем изображениям внутри папки и проверяет, является ли изображение на них птицей (выполнено функцией not_a_bird_check()).
    Args:
        dir_to_control (str): Путь к папке с фотографиями которые нужно прверить. 
    
    Return:    
        Возвращяет первоначальное количество файлов с изображениями и количество из них не является птицей
    """

    
    
    bird_check_start=time.time()
    files_paths = get_image_files(dir_to_control)
    tot=len(files_paths)
    not_birds_cnt=0
    for ind,file in enumerate(files_paths):    
        not_birds_cnt=not_birds_cnt+not_a_bird_check(file,dir_to_dump,ind+1)
        if ind%500==0:  # каждые 500 файлов - выдаем сообщение, чтобы следить за ходом выполнения проверки и сильно не заскучать
            print(f'Проверен файл {ind} из {len(files_paths)}. Времени с начала проверки {time.time()-bird_check_start}')
    return(tot,not_birds_cnt)


In [ ]:
# Функция, которая перебирает фотографии внутри папки, ищет и удаляет полные дубликаты
def delite_identical_files(dir1):
    """
        Пробегает по всем изображениям внутри папки и удаляет полные дубликаты
    Args:
        dir1 (str): Путь к папке с фотографиями которые нужно прверить. 
    
    Return:    
        Возвращяет первоначальное количество файлов с изображениями и количество удаленных файлов
    """


    
    start_deliting_doubles = time.time()
    images = get_image_files(dir1) #получаем список путей файлов
    images = images[::-1] ## меняем направление списка
    uniq_image={}
    ###---- Создаем список уникальных изображений
    for ind,file_path in enumerate( images):
        img= imread(file_path)# читаем файл как трехмерный массив
        img_list= tuple(img.flatten().tolist()) # переводим трехмерный массив в плоский список, а затем в кортеж
        uniq_image[img_list]=file_path # записываем словарь img_list:<путь к файлу>. Если у какого-то изображения далее будет такой-же img_list - значит они идентичны.
        if (ind+1)%1000==0: # Периодически выводим информацию о прогрессе проверки, чтобы пользователь сильно не заскучал. Делаем только раз в 1000 файлов(только на больших объемах проверки) 
            print(f'Проверено {ind+1} из {len(images)} файлов в папке {dir1} . Прошло времени от начала проверки папки  {time.time()-start_deliting_doubles } сек.')
    print(f'Всего файлов {len(images)} из них уникальных {len(uniq_image)}. Прошло времени от начала проверки папки {time.time()-start_deliting_doubles } сек.')
    uniq_images_list=uniq_image.values() 

    to_delite = [x for x in images if x not in uniq_images_list] # Создаем список дубликатов, которые планируем удалить
    for file_path in to_delite:
        os.remove(file_path)
    return(len(images),len(to_delite))

        
    


In [ ]:
# Функция удаляет нечитаемые картинки в папке
def verify_files(data_dir):
    """
        Проверяет все ли изображения в выбранной папке читаются. Если какие-то файлы с изображениями не читается - удаляет эти файлы
    Args:
        data_dir (str): Путь к директории с данными. 
    
    Return:    
        Возвращяет первоначальное количество файлов с изображениями и количество удаленных файлов
    """

    images=get_files(data_dir) # получаем перечень файлов
    failed = verify_images(images) # проверяем файлы и получаем перечень нечитаемых
    failed.map(Path.unlink) # удаляем нечитаемые файлы 
    return(len(images), len(failed))

In [86]:
# Основной код. Проводим последовательно три проверки:
#                                                     1) Читаются ли файлы?
#                                                     1) Нет ли дубликатов фотографий под разными именами?
#                                                     1) Являются ли фотографии, фотографиями птиц?

#
# Установка параметров
start = time.time() ## точка отсчета времени
dir_to_control="to_control" # Папка, которую проверяем
dir_to_dump="dump"  # Папка, в которую складываем файлы не удовлетворившие условию
сlass_dict ={'0':'птица',       # cловарь <класс>:<название класса> в модели ,bird_not_bird
             '1':'монета',
             '2':'человек',
             '5':'книга',
             '6':'рыба',
             '7':'авто',
             '8':'мотыльек',
             '9':'головастик'             
            }
model_path = 'bird_not_bird_04_02_25.pkl' # Путь к сохраненной модели
dir_to_check="to_control" # Папка, которую проверяем

os.makedirs(dir_to_dump, exist_ok=True)


# Проверяем все ли фотографии читаются
print(f'Проверяем, все ли файлы читаются',end=" ")
start_ver = time.time()                     # промежуточная точка отсчета времени
tot,delited=verify_files(dir_to_check)      #запускаем функцию проверки читаемости файлов
print(f'Всео файлов {tot}, из них удалено {delited}. Ушло {time.time()-start_ver} сек.',end="\n")




# Проверяем нет ли дубликатов. Проверку будем проводить по папкам классов
print(f'Проверяем на наличие дубликатов')
start_dubs_check = time.time()                  # промежуточная точка отсчета времени
list_subdir= [os.path.join(dir_to_control, name) for name in os.listdir(dir_to_control) if os.path.isdir(os.path.join(dir_to_control, name))] ## создаем список подкатегорий первого уровня в контролируемой папке
for subdir in list_subdir:
    start_dubs_check_dir = time.time()                  # промежуточная точка отсчета времени
    tot,delited=delite_identical_files(subdir)
    print(f'Всео файлов в папке {subdir} - {tot}. Из них удалено {delited}, так как дубликаты. Ушло на папку {time.time()-start_dubs_check_dir} сек.',end="\n")

print(f'Всео на прверку дубликатов ушло {time.time()-start_dubs_check } сек.',end="\n")




# Запуск проверки AI моделью на то, что все фотографии в папке - фотографии птиц
start_birds_check = time.time()                  # промежуточная точка отсчета времени
print(f'Проверяем, все ли фотографии - птицы.')
bird_not_bird_learner = load_learner(model_path) # Загружаем модель
dir_to_dump="dump"                               # Папка, в которую складываем файлы не удовлетворившие условию
os.makedirs(dir_to_dump, exist_ok=True)          # создаем категорию для отбракованых фотографий

tot,not_birds_cnt=birds_check(dir_to_check)      #запускаем функцию проверки на птиц
print(f'Всео проверено файлов {tot}, из них не птиц - {not_birds_cnt}. Ушло {time.time()-start_birds_check} сек.',end="\n")


# Итоговое сообщение
print(f'Все проверки пройдены. Ушло {time.time()-start} сек.',end="\n")


Проверяем, все ли файлы читаются Всео файлов 1346, из них удалено 0. Ушло 14.308270692825317 сек.
Проверяем на наличие дубликатов
Всего файлов 409 из них уникальных 409. Прошло времени от начала проверки папки 2.415631055831909 сек.
Всео файлов в папке to_control\0 - 409. Из них удалено 0, так как дубликаты. Ушло на папку 3.2430202960968018 сек.
Всего файлов 9 из них уникальных 9. Прошло времени от начала проверки папки 0.12535595893859863 сек.
Всео файлов в папке to_control\0 — на запрос дикуша птица - 9. Из них удалено 0, так как дубликаты. Ушло на папку 0.13656926155090332 сек.
Всего файлов 474 из них уникальных 474. Прошло времени от начала проверки папки 3.692931890487671 сек.
Всео файлов в папке to_control\1 - 474. Из них удалено 0, так как дубликаты. Ушло на папку 3.724260091781616 сек.
Всего файлов 421 из них уникальных 421. Прошло времени от начала проверки папки 3.0447981357574463 сек.
Всео файлов в папке to_control\2 - 421. Из них удалено 0, так как дубликаты. Ушло на папку 

Проверен файл 0 из 1313. Времени с начала проверки 0.34616661071777344


Проверен файл 500 из 1313. Времени с начала проверки 65.73621797561646


Проверен файл 1000 из 1313. Времени с начала проверки 134.41349005699158


Всео проверено файлов 1313, из них не птиц - 0. Ушло 176.453284740448 сек.
Все проверки пройдены. Ушло 201.72746109962463 сек.
